In [1]:
import sys, os
if 'google.colab' in sys.modules:
    %cd
    % rm -rf MPyDATA
    ! git clone --recurse-submodules -j8 https://github.com/Michaeldz36/MPyDATA.git
    %cd MPyDATA
    ! git checkout develop
    ! pip install -U $(cat requirements.txt | cut -d '=' -f 1)
else:
    sys.path.append(os.path.join(os.getcwd(), '../..'))

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from MPyDATA_examples.Olesik_et_al_2020.setup import Setup, default_opt_set
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3, x_log_of_pn
from MPyDATA.options import Options
from MPyDATA_examples.Olesik_et_al_2020.simulation import Simulation
from MPyDATA_examples.utils.show_plot import show_plot
from joblib import Parallel, parallel_backend, delayed
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA.arakawa_c.discretisation import discretised_analytical_solution
from MPyDATA_examples.utils.error_norms import L2, Smolarkiewicz_Grabowski_1990_eq21
from MPyDATA_examples.Olesik_et_al_2020.convergence_plotter  import polar_plot

In [3]:
GCs = np.linspace(.05,.95, 5)
nrs = np.array([2**n for n in range(10, 14)], dtype=int)

In [4]:
grid_layout = x_p2()
psi_coord = x_p2()
setup = Setup()

In [5]:
def analysis(setup, GC, opt):
    options = Options(**opt)
    simulation = Simulation(setup, grid_layout,  psi_coord, options, GC)
    simulation.step(simulation.out_steps[-1])
    t = simulation.out_steps[-1] * simulation.dt
    rh = simulation.rh
    pdf_t = PdfEvolver(setup.pdf, setup.drdt, t)
    def pdf_arg(r):
        return pdf_t(r* rh.units).magnitude
    analytical = discretised_analytical_solution(
                simulation.rh.magnitude,
                pdf_arg, midpoint_value = True
            ) * pdf_t(rh[0]).units
    numerical = simulation.n_of_r
    
    maximum_numeric = np.max(numerical)
    maximum_analytic = np.max(analytical)
    measure_height = (maximum_numeric / maximum_analytic).magnitude
    
    dif = analytical-numerical
    measure_h_inf = ((maximum_numeric - maximum_analytic) / maximum_analytic).magnitude
    measure_h_2 =  (1/t * np.sqrt( 1/len(dif) * dif.dot(dif))).magnitude
    

    def rel_disp(r, pdf):
        rd = 0
        for i in range(len(pdf)):
            mean = pdf[i] * (r[i+1] - r[i])
            std = pdf[i] * (r[i+1] - r[i])**2
            rd += std/mean
        return rd
    print("analytic rd", rel_disp(rh, analytical))
    print("numeric rd", rel_disp(rh, numerical))
    #TODO: with this kind of definition pdf cancels out

    
    error = np.log2(Smolarkiewicz_Grabowski_1990_eq21(numerical.magnitude, analytical.magnitude, t.magnitude))
    return setup.nr, GC, error, measure_height, measure_h_inf, measure_h_2 #, rel_dispersion

In [6]:
opts = default_opt_set
def replace_names(opt):
    str_repl = [["'n_iters': 1","upwind"],
                ["'n_iters': 2","MPDATA 2 iterations"],
                ["'n_iters': 3","MPDATA 3 iterations"],
                ["'",""],
                [": True",""],
                ["_", " "],
                ["{",""],["}",""],[","," "]]                            
    for repl in str_repl:
        opt = str(opt).replace(repl[0], repl[1])
    return opt

In [ ]:
for opt in opts:
    with parallel_backend('threading', n_jobs=-2):
        results0 = Parallel(verbose=10)(
            delayed(analysis)(Setup(nr = nr, mixing_ratios_g_kg = [1.05,]), GC, opt)
            for nr in nrs
            for GC in GCs
        )
    results = tuple(tuple(i) for i in zip(*results0))
    plot_setup = np.array(results[0:2])
    measures = {'log$_2$(err)':results[2] , 'height difference':results[3], 'h_inf':results[4], 'h_2': results[5]} #,'rel_disp':results[6]}
    opt = replace_names(opt)
    print(opt)
    for measure in measures.keys():
        polar_plot(nrs, GCs, measures[measure], name = measure)
        show_plot(filename = f'convergence_{measure}_{opt}.pdf')   #TODO: dont show plot name when savingfig (for masters)

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.


analytic rd nan micrometer
numeric rd 24.0 micrometer


C:\Users\Michael\anaconda3\lib\site-packages\pint\quantity.py:1160: RuntimeWarning: invalid value encountered in double_scalars
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


analytic rd nan micrometer
numeric rd 24.0 micrometer


C:\Users\Michael\anaconda3\lib\site-packages\pint\quantity.py:1160: RuntimeWarning: invalid value encountered in double_scalars
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


...

[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  1.2min
C:\Users\Michael\anaconda3\lib\site-packages\pint\quantity.py:1160: RuntimeWarning: invalid value encountered in double_scalars
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


analytic rd nan micrometer
numeric rd 24.0 micrometer


C:\Users\Michael\anaconda3\lib\site-packages\pint\quantity.py:1160: RuntimeWarning: invalid value encountered in double_scalars
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


analytic rd nan micrometer
numeric rd 24.0 micrometer


C:\Users\Michael\anaconda3\lib\site-packages\pint\quantity.py:1160: RuntimeWarning: invalid value encountered in double_scalars
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)
